## Написание алгоритма деревьев решений

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
from sklearn.datasets import load_iris

Выбрал датасет цветков ириса для ревлизации алгоритма, весь код специализирован исключительно для него  
иначе его прдется дорабатывать)

Заранее перемешиваем строки, чтобы всевозможные классы попали и в тренировочную выборку, и в тестовую

In [4]:
data = pd.DataFrame(load_iris()['data'])
data.columns = load_iris()['feature_names']
data['target'] = load_iris()['target']
data = data.sample(frac=1).reset_index(drop=True)
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.6,3.0,4.1,1.3,1
1,5.0,3.2,1.2,0.2,0
2,6.1,2.9,4.7,1.4,1
3,5.1,3.8,1.5,0.3,0
4,6.4,2.8,5.6,2.2,2
...,...,...,...,...,...
145,7.1,3.0,5.9,2.1,2
146,5.4,3.0,4.5,1.5,1
147,5.0,3.5,1.3,0.3,0
148,5.2,2.7,3.9,1.4,1


Функции разделения выборок

In [260]:
def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

def HGini(l):
    length = len(l)
    cnt = Counter(l)
    
    gini = 0
    for cl in cnt.values():
        p_1 = cl / length
        p_2 = (1 - p_1)
        it = p_1 * p_2
        gini += it
    
    return gini

def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

Здесь строки (в тренировочном датасете) были отсортированы по классам, в противном случае программа  
становится намного более громоздкой, не вижу смысла не делать этого)

In [218]:
data_train = data.loc[:0.7*len(data), :]
data_train = data_train.sort_values('target')
data_test = data.loc[0.7*len(data):, :]

In [301]:
# Функция определения входных даных, сначала подается таблица pandas, следующии итерации обрабатывают словари
# Задаются выборки для деления по классам
def step_1(data):
    
    if isinstance(data, dict):
        try:
            x = data['values']['first']['value']
            indexes = data['values']['first']['index']
            return cycler(x, indexes)
        except:
            try:
                x = data['values']['second']['value']
                indexes = data['values']['second']['index']
                return cycler(x, indexes)
            except:
                pass
    elif isinstance(data, pd.DataFrame):
        step = []
        x = data.target.values
        indexes = data.index.values
        return cycler(x, indexes)
    else:
        pass
    
# Данная функция делит выборку по принципу максимальной энтропии, строки по разделу определяет в два разных словаря,
# сохраняя информацию об индексе строки в датасете
def cycler(x, indexes):    
    for _ in range(len(x)):
#         Определяем максимальную энтропию в выборке    
        sepal_l = []
        for i in range(len(x)):
            left_l = x[:i]
            right_l = x[i:]
            sepal_l.append(HEntropy(x) - (len(left_l) / len(x)) * HEntropy(left_l) - (len(right_l) / len(x)) * HEntropy(right_l))
#         Находим индекс перегородки
        for n, el in enumerate(sepal_l):
            if el == max(sepal_l):
                baffle = n
                break
#                 Создаем словарь с вложенными словарями со строками отделенными по соотв. классу
    if len(set(x)) > 1:
        class_ = x[2]
        step = {'values': {'first': {'value': x[:baffle], 'index': indexes[:baffle]},\
                           'second':{'value': x[baffle:], 'index': indexes[baffle:]}}}
        step['values']['first']['class_'] = step['values']['first']['value'][0]
        step['values']['second']['class_'] = step['values']['second']['value'][0]
    else:
        step = {'values': {'first':{'value': x[:baffle], 'index': indexes[:baffle], 'class_': class_}}}
        step['values']['first']['class_'] = step['values']['first']['value'][0]
        return next_step(step, data_range)

    return step
    
# В этой функции сбрасывает из словар необходимую информацию в список data_range, при условии, что разбиении полностью
# выполнено
def next_step(step, data_range):
    try:
        if len(set(step['values']['first']['value'])) == 1:

            data_range.append({'1': data.iloc[step['values']['first']['index'], :1].values.reshape\
            (1, len(step['values']['first']['index']))[0], 'class_': step['values']['first']['class_']})

            data_range.append({'2': data.iloc[step['values']['first']['index'], 1:2].values.reshape\
            (1, len(step['values']['first']['index']))[0], 'class_': step['values']['first']['class_']})

            data_range.append({'3': data.iloc[step['values']['first']['index'], 2:3].values.reshape\
            (1, len(step['values']['first']['index']))[0], 'class_': step['values']['first']['class_']})

            data_range.append({'4': data.iloc[step['values']['first']['index'], 3:4].values.reshape\
            (1, len(step['values']['first']['index']))[0], 'class_': step['values']['first']['class_']})

            del step['values']['first']
    except:
        pass
    try:
        if len(set(step['values']['second']['value'])) == 1:

            data_range.append({'1': data.iloc[step['values']['second']['index'], :1].values.reshape\
            (1, len(step['values']['second']['index']))[0], 'class_': step['values']['second']['class_']})

            data_range.append({'2': data.iloc[step['values']['second']['index'], 1:2].values.reshape\
            (1, len(step['values']['second']['index']))[0], 'class_': step['values']['second']['class_']})

            data_range.append({'3': data.iloc[step['values']['second']['index'], 2:3].values.reshape\
            (1, len(step['values']['second']['index']))[0], 'class_': step['values']['second']['class_']})

            data_range.append({'4': data.iloc[step['values']['second']['index'], 3:4].values.reshape\
            (1, len(step['values']['second']['index']))[0], 'class_': step['values']['second']['class_']})

            del step['values']['second']
    except:
        pass

    return data_range, step

In [304]:
# Не получилось ока что вполнить вызовы функции внутри итератора, поэтому прописал необходимое кол-во вручную
# (частный случай для датасета iris)
data_range = []
step = step_1(data_train)

data_range, step = next_step(step, data_range)

step = step_1(step)

data_range, step = next_step(step, data_range)


data_range
# Значение ключа соответствует номеру столбца


[{'1': array([4.9, 5.4, 5. , 5.1, 4.8, 4.7, 5.5, 5.1, 4.8, 4.6, 5.1, 4.6, 4.9,
         5.7, 5.4, 4.8, 4.3, 4.5, 4.9, 5.2, 5.7, 5.1, 5.1, 4.9, 4.8, 5. ,
         5.2, 5.4, 4.6, 5.3, 5. , 5.4, 5.4, 4.4, 5.1, 4.4, 5.8]),
  'class_': 0},
 {'2': array([3.6, 3.9, 3.4, 3.8, 3.4, 3.2, 3.5, 3.3, 3.1, 3.6, 3.8, 3.4, 3. ,
         3.8, 3.7, 3.4, 3. , 2.3, 3.1, 3.5, 4.4, 3.5, 3.8, 3.1, 3. , 3.4,
         4.1, 3.9, 3.1, 3.7, 3.2, 3.4, 3.4, 3.2, 3.5, 2.9, 4. ]),
  'class_': 0},
 {'3': array([1.4, 1.3, 1.5, 1.6, 1.6, 1.3, 1.3, 1.7, 1.6, 1. , 1.9, 1.4, 1.4,
         1.7, 1.5, 1.9, 1.1, 1.3, 1.5, 1.5, 1.5, 1.4, 1.5, 1.5, 1.4, 1.6,
         1.5, 1.7, 1.5, 1.5, 1.2, 1.5, 1.7, 1.3, 1.4, 1.4, 1.2]),
  'class_': 0},
 {'4': array([0.1, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.5, 0.2, 0.2, 0.4, 0.3, 0.2,
         0.3, 0.2, 0.2, 0.1, 0.3, 0.2, 0.2, 0.4, 0.3, 0.3, 0.1, 0.1, 0.4,
         0.1, 0.4, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2]),
  'class_': 0},
 {'1': array([5. , 5.5, 6.6, 7. , 5.5, 6.1, 5.5, 5.6, 5. , 6

In [306]:
# Выделяем ветки
branches = {}
for n in (0, 4, 8):
    branches[f'{(n//4)+1} branch'] = {'1': {'data': data_range[n]['1'], 'class_': data_range[n]['class_']},\
                                 '2': {'data': data_range[n+1]['2'], 'class_': data_range[n+1]['class_']},\
                                 '3': {'data': data_range[n+2]['3'], 'class_': data_range[n+2]['class_']},\
                                 '4': {'data': data_range[n+3]['4'], 'class_': data_range[n+3]['class_']}}
branches

{'1 branch': {'1': {'data': array([4.9, 5.4, 5. , 5.1, 4.8, 4.7, 5.5, 5.1, 4.8, 4.6, 5.1, 4.6, 4.9,
          5.7, 5.4, 4.8, 4.3, 4.5, 4.9, 5.2, 5.7, 5.1, 5.1, 4.9, 4.8, 5. ,
          5.2, 5.4, 4.6, 5.3, 5. , 5.4, 5.4, 4.4, 5.1, 4.4, 5.8]),
   'class_': 0},
  '2': {'data': array([3.6, 3.9, 3.4, 3.8, 3.4, 3.2, 3.5, 3.3, 3.1, 3.6, 3.8, 3.4, 3. ,
          3.8, 3.7, 3.4, 3. , 2.3, 3.1, 3.5, 4.4, 3.5, 3.8, 3.1, 3. , 3.4,
          4.1, 3.9, 3.1, 3.7, 3.2, 3.4, 3.4, 3.2, 3.5, 2.9, 4. ]),
   'class_': 0},
  '3': {'data': array([1.4, 1.3, 1.5, 1.6, 1.6, 1.3, 1.3, 1.7, 1.6, 1. , 1.9, 1.4, 1.4,
          1.7, 1.5, 1.9, 1.1, 1.3, 1.5, 1.5, 1.5, 1.4, 1.5, 1.5, 1.4, 1.6,
          1.5, 1.7, 1.5, 1.5, 1.2, 1.5, 1.7, 1.3, 1.4, 1.4, 1.2]),
   'class_': 0},
  '4': {'data': array([0.1, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.5, 0.2, 0.2, 0.4, 0.3, 0.2,
          0.3, 0.2, 0.2, 0.1, 0.3, 0.2, 0.2, 0.4, 0.3, 0.3, 0.1, 0.1, 0.4,
          0.1, 0.4, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2]),
   'class_': 0}},

In [284]:
# Производим обучение, здесь так как я понял, фактом принадлежности к классу будет означать попадание исследуемого значения
# в диапазон полученных при обучении (либо всех, либо какого то одного), если любое условие подходит, то и любой класс
# может быть назначен, а все условию не выполняются никогда, поэтому у меня вопрос, как решить?? =(
classes_memory = []
_ = 1
first_cond = min(branches[f'{_} branch']['1']['data']) <= data_test.iloc[i, 0] <= max(branches[f'{_} branch']['1']['data'])
second_cond = min(branches[f'{_} branch']['2']['data']) <= data_test.iloc[i, 1] <= max(branches[f'{_} branch']['2']['data'])
third_cond = min(branches[f'{_} branch']['3']['data']) <= data_test.iloc[i, 2] <= max(branches[f'{_} branch']['3']['data'])
fourth_cond = min(branches[f'{_} branch']['4']['data']) <= data_test.iloc[i, 3] <= max(branches[f'{_} branch']['4']['data'])

for i in range(len(data_test)):
    for _ in range(1, 4):
        if first_cond and second_cond and third_cond and fourth_cond:
            classes_memory.append(branches[f'{_} branch']['1']['class_'])
            break
        else:
            classes_memory.append('None')
            break

In [307]:
classes_memory

['None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None']